# Data search and calculation of spectral indices for subareas - Sentinel-2
## 1) Search full coverage for designated subarase

        
This code searches for Sentinel-2 satellite images covering specific subareas within a specified date range and cloud cover limit, then generates a text file listing the unique image sets per tile and a heatmap visualizing the coverage frequency of these images across the subareas


In [ ]:
from eodag import EODataAccessGateway 
import geopandas as gpd
from shapely.geometry import shape
from datetime import datetime
from collections import defaultdict
from itertools import combinations
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm

subareas_path = '/path/to/area_boundaries/subareas_clip_to_AOI0_4326.shp'  # subareas for searching 
tiles_path = '/path/to/area_boundaries/subareas_4326.shp'  # base for the heatmap
txt_path = '/path/to/text_file_save_location/textfile.txt'
heatmap_path = '/path/to/heatmap_save_location/heatmap.png'

def parse_date(image_title):
    date_str = image_title.split('_')[2][:8]
    return datetime.strptime(date_str, '%Y%m%d')

subareas_data = gpd.read_file(subareas_path)
tiles_data = gpd.read_file(tiles_path)

dag = EODataAccessGateway()
product_type = "S2_MSI_L2A"
cloud_cover = 5
start_date = "2022-01-01"
end_date = "2022-12-31"

tile_coverage_sets = defaultdict(int)
coverage_results = defaultdict(list)

for _, row in subareas_data.iterrows():
    selected_tile_name = row['layer']
    tile_geometry = row.geometry
    search_results = dag.search_all(productType=product_type, geom=tile_geometry, start=start_date, end=end_date, cloudCover=cloud_cover)

    images_by_date = defaultdict(list)
    for product in search_results:
        image_date = parse_date(product.properties['title'])
        images_by_date[image_date].append(product)

    for date, products in images_by_date.items():
        full_coverage_found = False
        for product in products:
            product_geometry = shape(product.geometry)
            if product_geometry.contains(tile_geometry):
                coverage_results[selected_tile_name].append(product.properties['title'])
                full_coverage_found = True
                tile_coverage_sets[selected_tile_name] += 1
                break
        
        if not full_coverage_found:
            for combo in combinations(products, 2):
                combined_geometry = shape(combo[0].geometry).union(shape(combo[1].geometry))
                if combined_geometry.contains(tile_geometry):
                    coverage_results[selected_tile_name].extend([p.properties['title'] for p in combo])
                    tile_coverage_sets[selected_tile_name] += 1
                    break

unique_dates_per_tile = defaultdict(set)
for tile, titles in coverage_results.items():
    for title in titles:
        date_of_image = parse_date(title)
        unique_dates_per_tile[tile].add(date_of_image)

with open(txt_path, 'w') as file:
    for tile, dates in unique_dates_per_tile.items():
        file.write(f"{tile}: {len(dates)} unique image sets\n")
        for date in sorted(dates):
            date_str = date.strftime('%Y %m %d')
            titles = [title for title in coverage_results[tile] if parse_date(title) == date]
            file.write(f"  Image ({date_str}): {', '.join(titles)}\n")


tiles_data['num_images'] = tiles_data['layer'].map(tile_coverage_sets).fillna(0)
num_images_bins = list(range(11))  
norm = BoundaryNorm(num_images_bins, ncolors=len(num_images_bins) - 1, clip=True)
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
heatmap = tiles_data.plot(column='num_images', ax=ax, cmap='GnBu', edgecolor='black', vmin=0, vmax=10)
subareas_data.boundary.plot(ax=ax, color='black')

for idx, row in tiles_data.iterrows():
    plt.annotate(text=f"{int(row['num_images'])}", xy=(row.geometry.centroid.x, row.geometry.centroid.y),
                 horizontalalignment='center', verticalalignment='center')

plt.text(0.5, 1.05, f"Sentinel-2 Search Period: {start_date} to {end_date}", 
         ha='center', va='center', transform=ax.transAxes)

cbar = fig.colorbar(heatmap.collections[0], ax=ax, extend='neither')
cbar.set_label('Number of image sets')
cbar.set_ticks(range(0, 11)) 

plt.savefig(heatmap_path, dpi=500)

## 2) Calculation of spectral indices
       
This code processes satellite imagery by clipping spectral bands to specific subarea boundaries, merging images, and calculating various spectral indices for areas of interest defined in a text fil., 

***Images are searched from the image repository on the ARDF virtual machine

In [ ]:
import re
import os
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import datetime
import shutil
from rasterio.merge import merge
import subprocess
import numpy as np
import time

areas_to_search = ['AOI0_1', 'AOI0_2','AOI0_3','AOI0_4','AOI0_5','AOI0_6','AOI0_7','AOI0_8','AOI0_9','AOI0_10','AOI0_11','AOI0_12','AOI0_13','AOI0_14','AOI0_15','AOI0_16','AOI0_17','AOI0_18','AOI0_19','AOI0_20', 'AOI0_21', 'AOI0_22'] 

shp_file = '/path/to/area_boundaries/subareas_4326.shp'
txt_file = '/path/to/text_file_save_location/textfile.txt'    
output_base_folder = '/path/to/output_files_save_location/sentinel2/'

base_folder = '/initial_folder_for_image_search/'

# GENERATING IMAGE PATHS ON VM
def generate_image_paths(txt_file, base_folder, areas_to_search=None):
    with open(txt_file, 'r') as file:
        lines = file.readlines()

    image_paths = {}
    current_area = None
    for line in lines:
        if line.startswith('AOI0'):
            current_area = line.split(':')[0].strip()
            if areas_to_search is not None and current_area not in areas_to_search:
                current_area = None 
                continue
            image_paths[current_area] = []
        elif 'Image' in line and current_area:
            image_names = re.findall(r'S2[AB]_MSIL2A_\d{8}T\d{6}_N\d{4}_R\d{3}_T\d{2}[A-Z]{3}_\d{8}T\d{6}', line)
            cover_paths = []
            for image_name in image_names:
                product_identifier = generate_product_identifier(image_name)
                if product_identifier:
                    full_path = f'{base_folder}{product_identifier}'
                    cover_paths.append(full_path)
            if cover_paths:
                image_paths[current_area].append(cover_paths)

    return image_paths

def generate_product_identifier(image_name):
    # Generating productIdentifier based on image name
    match = re.match(r'(S2[AB])_MSIL2A_(\d{4})(\d{2})(\d{2})T.*', image_name)
    if match:
        satellite, year, month, day = match.groups()
        return f'/eodata/Sentinel-2/MSI/L2A/{year}/{month}/{day}/{image_name}.SAFE'
    return None

# GENERATING PATHS TO SPECTRAL BANDS OF IMAGES ON VM
def find_jp2_files_for_bands(image_path):
    band_paths = {"R20m": {"B02": None, "B03": None, "B04": None, "B8A": None, "B11": None, "B12": None}}
    for root, _, files in os.walk(image_path):
        for file in files:
            if file.lower().endswith(".jp2"):
                for band in band_paths["R20m"]:
                    if f"_{band.lower()}_" in file.lower() and "20m" in file.lower():
                        band_paths["R20m"][band] = os.path.join(root, file)
    return band_paths

def add_band_paths_to_images(image_paths_by_area):
    for area, cover_paths in image_paths_by_area.items():
        for cover in cover_paths:
            for img_path in cover:
                band_paths = find_jp2_files_for_bands(img_path)
                if band_paths:
                    img_path_index = cover.index(img_path)
                    cover[img_path_index] = {"image": img_path, "bands": band_paths}

# CLIPPING SPECTRAL BANDS TO SUBAREA BOUNDARIES                  
def get_subarea_bounds(shp_file, subarea, raster_crs):
    gdf = gpd.read_file(shp_file)
    if gdf.crs != raster_crs:
        gdf = gdf.to_crs(raster_crs)
    subarea_geom = gdf[gdf['layer'] == subarea].geometry
    return subarea_geom

def clip_and_save_raster(image_path, bounds, output_folder, band, img_date):
    with rasterio.open(image_path) as src:
        raster_crs = src.crs
        out_image, out_transform = mask(src, shapes=bounds, crop=True)
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

        output_path = os.path.join(output_folder, f"{band}_{img_date}_clipped.tif")
        with rasterio.open(output_path, "w", **out_meta) as dest:
            dest.write(out_image)

def extract_date_from_image_name(image_name):
    date_str = re.search(r'\d{8}', image_name).group()
    return datetime.datetime.strptime(date_str, '%Y%m%d').strftime('%Y-%m-%d')

def merge_images_with_gdal(image_paths, output_path):
    with open(os.devnull, 'w') as devnull:
        cmd = ["gdal_merge.py", "-o", output_path] + image_paths
        subprocess.call(cmd, stdout=devnull, stderr=subprocess.STDOUT)

# CALCULATING SPECTRAL INDICES
def calculate_all_indices(output_folder_tile, red_path, nir_path, green_path, blue_path, swir1_path, swir2_path, date, area):
    with rasterio.open(red_path) as red_src, rasterio.open(nir_path) as nir_src, rasterio.open(green_path) as green_src, rasterio.open(blue_path) as blue_src, rasterio.open(swir1_path) as swir1_src, rasterio.open(swir2_path) as swir2_src:
        red = red_src.read(1)
        nir = nir_src.read(1)
        green = green_src.read(1)
        blue = blue_src.read(1)
        swir1 = swir1_src.read(1)
        swir2 = swir2_src.read(1)
    
    red = red.astype(np.float32) / 65535.0
    nir = nir.astype(np.float32) / 65535.0
    green = green.astype(np.float32) / 65535.0
    blue = blue.astype(np.float32) / 65535.0
    swir1 = swir1.astype(np.float32) / 65535.0
    swir2 = swir2.astype(np.float32) / 65535.0

    np.seterr(divide='ignore', invalid='ignore')

    red[red == 0] = np.nan
    nir[nir == 0] = np.nan
    green[green == 0] = np.nan
    blue[blue == 0] = np.nan
    swir1[swir1 == 0] = np.nan
    swir2[swir2 == 0] = np.nan    
    
    ndvi = (nir - red) / (nir + red)
    gndvi = (nir - green) / (nir + green)
    evi = 2.5 * ((nir - red) / (nir + 6 * red - 7.5 * blue + 1))
    savi = (1 + 0.5) * ((nir - red) / (nir + red + 0.5)) 
    osavi = ((nir - red) / (nir + red + 0.16))
    dvi = nir - red
    sr = red / nir
    gemi = ((2 * (nir ** 2 - red ** 2) + 1.5 * nir + 0.5 * red) / (nir + red + 0.5)) * (1 - 0.25 * ((2 * (nir ** 2 - red ** 2) + 1.5 * nir + 0.5 * red) / (nir + red + 0.5))) - ((red - 0.125) / (1 - nir))
    ndwi = (green - nir) / (green + nir)
    mndwi = (green - swir1) / (green + swir1)
    lswi = (nir - swir1) / (nir + swir1)
    ui = (swir2 - nir) / (swir2 + nir)
    ndbi = (swir1 - nir) / (swir1 + nir)
    mndbi = (swir2 - blue) / (swir2 + blue)      
    bsi = ((red + swir1) - (nir + blue)) / ((red + swir1) + (nir + blue))                                                                          

    indices = {"NDVI": ndvi, "GNDVI": gndvi, "EVI": evi, "SAVI": savi, "OSAVI": osavi, "DVI": dvi, "SR": sr, "GEMI": gemi, "NDWI": ndwi, "BSI": bsi, "MNDBI": mndbi, "LSWI": lswi, "MNDWI": mndwi, "NDBI": ndbi, "UI": ui}

    for index_name, index_data in indices.items():
        output_file = os.path.join(output_folder_tile, f"S2_{area}_{index_name}_{date}.tif")

        with rasterio.open(output_file, 'w', driver='GTiff', width=red.shape[1], height=red.shape[0], count=1, dtype=str(index_data.dtype), crs=red_src.crs, transform=red_src.transform) as dst:
            dst.write(index_data, 1)
                        
# PROCESSING
def process_area_bands(area, cover_paths, shp_file, output_base_folder):
    start_time_subarea = time.time()

    for i, cover in enumerate(cover_paths, start=1):
        cover_date = extract_date_from_image_name(cover[0]["image"])
        num_images_in_cover = len(cover) 
        cover_folder = os.path.join(output_base_folder, f"{area}_{cover_date}_coverage_{i}")
        os.makedirs(cover_folder, exist_ok=True)

        first_band_path = list(cover[0]['bands']['R20m'].values())[0]
        if first_band_path:
            with rasterio.open(first_band_path) as src:
                raster_crs = src.crs
                bounds = get_subarea_bounds(shp_file, area, raster_crs)

        paths_for_bands = {band: [] for band in ["B02", "B03", "B04", "B8A", "B11", "B12"]}

        red_path = nir_path = green_path = blue_path = swir1_path = swir2_path = None

        if num_images_in_cover > 1:
            paths_for_bands = {band: [] for band in ["B02", "B03", "B04", "B8A", "B11", "B12"]}

            for img_info in cover:
                bands = img_info['bands']['R20m']
                img_date = extract_date_from_image_name(img_info["image"])
                first_band_path = list(bands.values())[0]
                if first_band_path:
                    with rasterio.open(first_band_path) as src:
                        raster_crs = src.crs
                        bounds = get_subarea_bounds(shp_file, area, raster_crs)

                    for band, band_path in bands.items():
                        if band_path:
                            sub_output_folder = os.path.join(cover_folder, os.path.basename(img_info["image"]))
                            os.makedirs(sub_output_folder, exist_ok=True)
                            clipped_path = os.path.join(sub_output_folder, f"{band}_{img_date}_clipped.tif")
                            clip_and_save_raster(band_path, bounds, sub_output_folder, band, img_date)
                            paths_for_bands[band].append(clipped_path)
                            
        for band, paths in paths_for_bands.items():
            merged_output_path = os.path.join(cover_folder, f"{band}_{cover_date}_merged.tif")
            merge_images_with_gdal(paths, merged_output_path)

            for band in ["B02", "B03", "B04", "B8A", "B11", "B12"]:
                band_path = os.path.join(cover_folder, f"{band}_{cover_date}_merged.tif")
                if band == "B02":
                    blue_path = band_path
                elif band == "B03":
                    green_path = band_path
                elif band == "B04":
                    red_path = band_path
                elif band == "B8A":
                    nir_path = band_path
                elif band == "B11":
                    swir1_path = band_path
                elif band == "B12":
                    swir2_path = band_path
        else:
            for img_info in cover:
                img_date = extract_date_from_image_name(img_info["image"])
                bands = img_info['bands']['R20m']
                for band, band_path in bands.items():
                    if band_path:
                        clipped_path = os.path.join(cover_folder, f"{band}_{img_date}_clipped.tif")
                        clip_and_save_raster(band_path, bounds, cover_folder, band, img_date)
                        paths_for_bands[band].append(clipped_path)
                        if band == "B02":
                            blue_path = clipped_path
                        elif band == "B03":
                            green_path = clipped_path
                        elif band == "B04":
                            red_path = clipped_path
                        elif band == "B8A":
                            nir_path = clipped_path
                        elif band == "B11":
                            swir1_path = clipped_path
                        elif band == "B12":
                            swir2_path = clipped_path

        for band, paths in paths_for_bands.items():
            merged_output_path = os.path.join(cover_folder, f"{band}_{cover_date}_merged.tif")
            merge_images_with_gdal(paths, merged_output_path)

        if all([red_path, nir_path, green_path, blue_path, swir1_path, swir2_path]):
            calculate_all_indices(cover_folder, red_path, nir_path, green_path, blue_path, swir1_path, swir2_path, img_date, area)

        for band_file in os.listdir(cover_folder):
            if band_file.startswith('B') and band_file.endswith('.tif'):
                os.remove(os.path.join(cover_folder, band_file))
                
    end_time_subarea = time.time()
    elapsed_time_subarea = round((end_time_subarea - start_time_subarea)/60, 3)
    print(f"Processing time for {area}: {elapsed_time_subarea} [min]")


start_time_whole_process = time.time()     
image_paths_by_area = generate_image_paths(txt_file, base_folder, areas_to_search)
add_band_paths_to_images(image_paths_by_area)

for area, cover_paths in image_paths_by_area.items():
    process_area_bands(area, cover_paths, shp_file, output_base_folder)

end_time_whole_process = time.time()
elapsed_time_whole_process = round((end_time_whole_process - start_time_whole_process)/60, 3)
print(f"Total processing time: {elapsed_time_whole_process} [min]")